# Distortion Correction

In [1]:
import numpy as np
import cv2
import glob
import pickle
import matplotlib.pyplot as plt

import matplotlib.image as mpimg
%matplotlib inline

#Load calibrated camera
Cam_Calib=pickle.load( open("CamCal/camCal_pickle.p", "rb"))

save_dir= r'output_images/pipeline/'

cam_dist=Cam_Calib["dist"]
cam_mtx=Cam_Calib["mtx"]

def undistort_s(img, dist=cam_dist, mtx=cam_mtx):
    '''Undistorts color image, ouputs undistorted img, saturation channel'''
    dst=cv2.undistort(img, cam_mtx, cam_dist, None, cam_mtx)
    HLS_img=cv2.cvtColor(dst, cv2.COLOR_RGB2HLS)
    S_channel= HLS_img[:,:,2]
    return dst, S_channel


# Gradient Threshold

In [2]:
#Diagonal sobel kernels from http://homepages.inf.ed.ac.uk/rbf/HIPR2/linedet.htm
up_kernel = [[-1.,-1.,2.],
         [-1.,2.,-1.],
         [2.,-1.,-1.]]
up_kernel=np.asarray(up_kernel)/12.0

down_kernel = [[2.,-1.,-1.],
         [-1.,2.,-1.],
         [-1.,-1.,2.]]
down_kernel=np.asarray(down_kernel)/12.0

def abs_sobel_thresh(gray, orient='x', thresh_min=0, thresh_max=255, sobel_kernel=3):
    """Axis sobel, asumes img is gray image, returns binary image"""
    # Apply the following steps to img
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient=='x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    elif orient=='y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    elif orient=='+':
        sobel = cv2.filter2D(gray,-1,up_kernel)
    elif orient=='-':
        sobel = cv2.filter2D(gray,-1,down_kernel)
    else:
        raise "Not valid orientation!"
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel=np.uint8(255 * abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
    binary_output=np.zeros_like(scaled_sobel)
            # is > thresh_min and < thresh_max
    # 6) Return this mask as your binary_output image
    binary_output[(scaled_sobel>thresh_min) & (scaled_sobel<thresh_max)] = 1
    return binary_output

def hls_tresh(S_channel, low_limit=0, high_limit=255):
    """Takes saturation channel, outputs binary image"""
    s_binary=np.zeros_like(S_channel)
    s_binary[(S_channel>=low_limit) & (S_channel<=high_limit)]=1
    return s_binary

def multiple_threshold(image, S_channel, min_dt=0, max_dt=255, s_low=128, s_high=255, k_slope1=up_kernel, k_slope2=down_kernel):
    """Takes gray image and Saturation channel, outputs binary image"""
    # Apply the following steps to img
    # 1) Apply sobel in direction
    #X not used, noisy
    #Less significant on Y axis , lots of noise
    #Diagonal Sobels!
    pos=abs_sobel_thresh(image, orient='+', thresh_min=min_dt, thresh_max=max_dt)
    neg=abs_sobel_thresh(image, orient='-', thresh_min=min_dt, thresh_max=max_dt)
    bin_diag=np.zeros_like(pos)
    bin_diag[(pos==1)|(neg==1)]=1
    #Close gaps in binary image
    bin_diag=cv2.morphologyEx(bin_diag, cv2.MORPH_CLOSE, np.ones((7,3),np.uint8))
#    
    #Even smaller magnitudes cause noise, not using magnitude sobel
    #Lots of noise (even when combining magnitude and orientation), not using orientation sobel
    # 4) Saturation channel
    s_bin= hls_tresh(S_channel ,low_limit=s_low, high_limit=s_high)
    #Open big spaces, as to desaturate the saturated channel (bad_joke_here)
    s_bin=cv2.morphologyEx(s_bin, cv2.MORPH_OPEN, np.ones((1,3),np.uint8))   
    # 5)Mix 2 above
    binary_output=np.zeros_like(bin_diag)
    binary_output[((bin_diag==1)|(s_bin==1))]=1
    #binary_output=bin_diag
    binary_output= cv2.erode(binary_output,np.ones((1,3),np.uint8),iterations = 1)
    return binary_output

# Prespective transform

In [3]:
#Polygon borders (Original IMG)
src_pts = np.array([[230,700],[610,440],[670,440],[1080,700]], np.float32)
#Destination Points
dst_pts = np.array([[300,720],[300,-200],[950,-200],[950,720]], np.float32)

def birds_eye(img, src=src_pts, dst=dst_pts):
    """Transform image to birdseye"""
    M = cv2.getPerspectiveTransform(src, dst)
    bird = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_NEAREST)
    return bird

# Line detect

In [4]:
window_width= 150
window_height=20
margin=40

def window_mask(width, height, img_ref, center, level):
    """Get window mask"""
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output
    
def find_window_centroids(warped, window_width, window_height, margin, custom_window=None):
    """Find centroids of windows in image, returns windows centroids and central point of lanes"""
    # Add here condition for custom distribution (not implemented)
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    #Custom window, triangular shape as to push the weight of the convolution to the center
    for i in range(int(len(window)/2)):
        window[i]=window[i]+i
        window[-i]=window[-i]+i
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(warped[int(warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    #Center of both lane markings
    center=r_center-l_center
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer, mode='full')
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
        max_conv = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        if conv_signal[int(max_conv)]>0:
            l_center=max_conv
        else:
            l_center=l_center
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
        max_conv = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        if conv_signal[int(max_conv)]>0:
            r_center=max_conv
        else:
            r_center=r_center
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))
    return window_centroids, center

def regression(window_centroids, window_height):
    """Get polinomial regression for left and right lanes"""
    pt_l=[]
    pt_r=[]
    x_point=[]
    for i, point in enumerate(window_centroids):
        pt_l.append((point[0]))
        pt_r.append((point[1]))
        x_point.append(720-i*window_height)
    fit_l=np.polyfit(x_point, pt_l, 2)
    fit_r=np.polyfit(x_point, pt_r, 2)    
    return fit_l, fit_r

def draw_regression(warped, window_centroids, window_height):
    """Draws regression on image"""
    if len(window_centroids) > 0:
        #Get polinomials
        left_fit, right_fit = regression(window_centroids, window_height)
        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)
        #get pixels for regression (currently unused)
        nonzero=warped.nonzero()
        nonzerox=np.array(nonzero[0])
        nonzeroy=np.array(nonzero[1])
        # Go through each level and draw the windows 
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255
        # Draw windows        
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channle 
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage = np.array(cv2.merge((255*warped,255*warped,255*warped)),np.uint8) # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
        #Draw polinomial lines
        ploty =np.linspace(0, warped.shape[0]-1, warped.shape[0])
        left_fitx=(left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2])
        right_fitx = (right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2])
        left_line=[]
        right_line=[]
        for y,l,r in zip(ploty, left_fitx, right_fitx):
            left_line.append([np.int32(l), np.int32(y)])
            right_line.append([np.int32(r), np.int32(y)])
        pts1=np.array(left_line, np.int32)
        pts2=np.array(right_line, np.int32)
        pts1.reshape((-1,1,2))
        pts2.reshape((-1,1,2))
        output=cv2.polylines(output, [pts1, pts2], False, (255,0,0), 3)
    # If no window centers found, just display orginal road image and empty polynom
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)
        right_fit=[]
        left_fit=[]
    return output, left_fit, right_fit

def tangent_circle(poly_l, poly_r, pic_center, eval_point=720):
    """Takes polynomial regressions and lanes center, returns curvature and distance to center [m]"""
    #pixels to meter ratio
    ym_per_pix = 3.0/106
    xm_per_pix = 3.7/650
    ploty = np.linspace(0, 719, num=720)
    leftx=np.array([poly_l[0]*(y**2)+poly_l[1]*(y)+poly_l[0] for y in ploty])
    rightx=np.array([poly_r[0]*(y**2)+poly_r[1]*(y)+poly_r[0] for y in ploty])
    
    center=(pic_center-700)*xm_per_pix

    poly_ml=np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    poly_mr=np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    
    eval_point*=ym_per_pix
    
    curv_l = ((1 + (2*poly_ml[0]*eval_point + poly_ml[1])**2)**1.5) / np.absolute(2*poly_ml[0])
    curv_r = ((1 + (2*poly_mr[0]*eval_point + poly_mr[1])**2)**1.5) / np.absolute(2*poly_mr[0])
    
    return curv_l, curv_r, center

def unwarp_color(warped_color, left_fit, right_fit, text, src=dst_pts, dst=src_pts):
    """Takes color image, returns identified poly, unwarps to original"""
    #Draw polinomial lines
    ploty =np.linspace(0, warped_color.shape[0]-1, warped_color.shape[0])
    left_fitx=(left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2])
    right_fitx = (right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2])
    left_line=[]
    right_line=[]
    for y,l,r in zip(ploty, left_fitx, right_fitx):
        left_line.append([np.int32(l), np.int32(y)])
        right_line.append([np.int32(r), np.int32(y)])
    pts1=np.array(left_line, np.int32)
    pts2=np.array(right_line, np.int32)
    pts1.reshape((-1,1,2))
    pts2.reshape((-1,1,2))
    total_pts=[]
    total_pts.append(pts1)
    total_pts.append(list(reversed(pts2)))
    total_pts=np.array(total_pts).reshape((-1,1,2))
    curvature_txt="Curvature: "+str(text[0])[:5] +"[m]"
    center_txt="Distance to lane Center: "+str(text[2])[0:5]+"[m]"
    output=np.zeros_like(warped_color)
    output=cv2.fillPoly(output, [total_pts], color=(255,0,255))
    output=cv2.polylines(output, [pts1, pts2], False, (255,0,0), 20)
    M = cv2.getPerspectiveTransform(src, dst)
    output = cv2.warpPerspective(output, M, (warped_color.shape[1], warped_color.shape[0]), flags=cv2.INTER_NEAREST)
    output = cv2.putText(output,curvature_txt, (50,50), cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255,255,255))
    output = cv2.putText(output,center_txt, (50,100), cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255,255,255))
    return output
    
    

In [5]:
#Full pipeline
def sanity():
    sane=True
    return sane

def pipeline(camera_image, window_width=window_width, window_height=window_height, margin=margin, save_step=False, save_dir=r'output_images/pipeline/', prefix='0'):
    image, S_channel=undistort_s(camera_image)
    gray=cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray=cv2.blur(gray,(3,3))
    bin_im=multiple_threshold(gray, S_channel)
    warp= birds_eye(bin_im)
    warp_color=birds_eye(image)
    window_centroids, pic_center = find_window_centroids(warp, window_width, window_height, margin)
    detect_lines, left_fit, right_fit =draw_regression(warp, window_centroids, window_height)
    curv_l, curv_r, center = tangent_circle(left_fit, right_fit, pic_center)
    text_array=[curv_l, curv_r, center]
    detect_color=unwarp_color(warp_color, left_fit, right_fit, text_array)
    result=cv2.addWeighted(image, 1, detect_color, .8, 0)
    if save_step:
        cv2.imwrite(save_dir+prefix+"undist.jpg", image)
        cv2.imwrite(save_dir+prefix+"gray.jpg", gray)
        cv2.imwrite(save_dir+prefix+"binary.jpg", 255*bin_im)
        cv2.imwrite(save_dir+prefix+"S_channel.jpg", 255*S_channel)
        cv2.imwrite(save_dir+prefix+"warp_color.jpg", warp_color)
        cv2.imwrite(save_dir+prefix+"warp.jpg", 255*warp)
        cv2.imwrite(save_dir+prefix+"detected.jpg", detect_lines)
        cv2.imwrite(save_dir+prefix+"detected_color.jpg", detect_color)
        cv2.imwrite(save_dir+prefix+"result.jpg", result)
    return result, left_fit, right_fit, text_array

def frame_only(camera_image):
    img, left_fit, right_fit, text_array = pipeline(camera_image)
    return img

In [249]:
img_files = glob.glob('test_images/*.jpg')
history=[]
for i, file in enumerate(img_files):
    camera_image=cv2.imread(file)
    history.append(pipeline(camera_image, save_step=True, prefix=str(i))[1:])
    

In [6]:
#for i in history:
#    print (i)

#Video
from moviepy.editor import VideoFileClip
#from IPython.display import HTML

video_output="Test.mp4"
clip1=VideoFileClip("project_video.mp4")
video_clip=clip1.fl_image(frame_only)
%time video_clip.write_videofile(video_output, audio=False)



[MoviePy] >>>> Building video Test.mp4
[MoviePy] Writing video Test.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [06:21<00:00,  3.34it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Test.mp4 

Wall time: 6min 22s
